In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from sklearn.neighbors import KNeighborsRegressor


from warnings import simplefilter

rand_st=4
#simplefilter("ignore") 

In [2]:
#output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx')
output=pd.read_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_standard.xlsx')
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986714,431.864722,0.458002,0.174,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.169,Обучение на разнице в рейтинге


In [3]:
#elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
#features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
features=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\features_scale.csv')

print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 224 entries, 0 to 223
dtypes: float64(224)
memory usage: 41.5 MB
None
(24255, 224)


,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
count,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,...,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04,2.425500e+04
mean,4.651173e-16,-5.095445e-17,2.800389e-17,1.662185e-16,-9.574787e-17,-5.084002e-17,-1.982198e-17,8.776964e-18,7.577713e-18,1.615671e-17,...,2.849339e-15,-4.317671e-16,-1.951599e-15,-3.907546e-16,3.863100e-16,3.976983e-16,6.822001e-17,-4.916519e-16,-5.504472e-16,1.367192e-16
std,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,...,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00
min,-1.304386e+00,-1.986583e+00,-9.384264e+00,-2.573847e+01,-5.354370e+00,-1.999656e+00,-5.434198e+00,-4.568270e-01,-1.073938e+00,-6.454322e-01,...,-7.743414e-01,-9.268193e-01,-7.653971e-01,-2.222981e+00,-9.405074e-01,-9.327001e-01,-1.388087e+00,-7.011194e-01,-3.486775e-01,-2.486221e-01
25%,-1.304386e+00,-6.931020e-01,-3.921183e-01,-2.674594e-02,-2.050809e-01,-7.069187e-01,1.470290e-01,-4.049769e-01,-6.064252e-01,-5.328262e-01,...,-7.743414e-01,-9.268193e-01,-7.653971e-01,-5.859648e-01,-9.405074e-01,-9.327001e-01,-1.388087e+00,-7.011194e-01,-3.486775e-01,-2.486221e-01
50%,-1.074322e-01,-1.387530e-01,-2.703187e-02,-2.674594e-02,-4.842544e-02,-1.528884e-01,3.384633e-01,-3.658697e-01,-3.103336e-01,-3.555585e-01,...,-7.743414e-01,-9.268193e-01,-7.653971e-01,-4.029267e-02,-9.405074e-01,5.447451e-01,-5.200025e-02,-3.532702e-01,-3.486775e-01,-2.486221e-01
75%,1.089521e+00,5.387848e-01,3.983948e-01,3.165405e-02,1.880723e-01,5.550391e-01,3.652735e-01,-1.268322e-01,2.350984e-01,-4.913152e-02,...,8.410065e-01,1.064147e+00,8.711250e-01,5.053794e-01,1.049028e+00,5.447451e-01,1.284087e+00,3.424282e-01,-3.486775e-01,-2.486221e-01
max,1.089521e+00,7.560540e+00,8.739280e+00,2.526270e+01,5.290408e+00,7.541976e+00,3.859691e-01,4.993140e+00,1.114374e+01,7.672212e+00,...,2.456354e+00,3.055113e+00,2.507647e+00,2.142396e+00,3.038562e+00,2.022190e+00,1.284087e+00,2.399617e+01,8.025472e+00,9.578986e+00


In [5]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [6]:
X_train_sum, X_test_sum, Y_train_sum, Y_test_sum =ms.train_test_split(features, target_sum, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_sum), len(Y_train_sum)))
print(X_train_sum.shape)
print(Y_train_sum.shape)
print(X_test_sum.shape)
print(Y_test_sum.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


In [7]:
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff =ms.train_test_split(features, target_diff, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_diff), len(Y_train_diff)))
print(X_train_diff.shape)
print(Y_train_diff.shape)
print(X_test_diff.shape)
print(Y_test_diff.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


#### Для теста

In [8]:
target_sum1=target_sum[:5000]
features1=features[:5000]
X_train_sum1, X_test_sum1, Y_train_sum1, Y_test_sum1 =ms.train_test_split(features1, target_sum1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_sum1), len(Y_train_sum1)))
print(X_train_sum1.shape)
print(Y_train_sum1.shape)
print(X_test_sum1.shape)
print(Y_test_sum1.shape)

Train data size: 3500 instances 
Test data size: 3500 instances
(3500, 224)
(3500,)
(1500, 224)
(1500,)


In [9]:
target_diff2=target_diff[:5000]
features2=features[:5000]
X_train_diff2, X_test_diff2, Y_train_diff2, Y_test_diff2 =ms.train_test_split(features2, target_diff2, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_diff2), len(Y_train_diff2)))
print(X_train_diff2.shape)
print(Y_train_diff2.shape)
print(X_test_diff2.shape)
print(Y_test_diff2.shape)

Train data size: 3500 instances 
Test data size: 3500 instances
(3500, 224)
(3500,)
(1500, 224)
(1500,)


In [10]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [11]:
def GridTune_sum(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))

    '''train_X_data='X_train_'+mode
    train_Y_data='Y_train_'+mode'''
    
    grid_best_model = grid.fit(X_train_sum1,Y_train_sum1)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [12]:
kn_sum_params =[{'n_neighbors':[2,5,10,15],
                'metric':['chebyshev','manhattan'],
                'leaf_size':[15,30,50]}]

start_kn_sum = time.time()

kn_sum1=KNeighborsRegressor(n_jobs=-1,weights='distance',algorithm='auto',)
kn_sum_model1=GridTune_sum(kn_sum_params,kn_sum1,5)
kn_sum_predictions1=kn_sum_model1.predict(X_test_sum1)

scores_kn_sum1=predictions_report(Y_test_sum1,kn_sum_predictions1)
full_time_kn_sum=round(time.time() - start_kn_sum,3)
print(full_time_kn_sum, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'leaf_size': 50, 'metric': 'chebyshev', 'n_neighbors': 2}

Лучший алгоритм:
KNeighborsRegressor(algorithm='auto', leaf_size=50, metric='chebyshev',
          metric_params=None, n_jobs=-1, n_neighbors=2, p=2,
          weights='distance')
MAE = 441.974
RMSE = 555.594
Correlation coefficient = 0.152
63.419 c


In [13]:
start_kn_sum = time.time()

kn_sum=KNeighborsRegressor(algorithm='auto', leaf_size=50, metric='chebyshev',
          metric_params=None, n_jobs=-1, n_neighbors=2, p=2,
          weights='distance').fit(X_train_sum,Y_train_sum)

#kn_sum_model=GridTune_sum(kn_sum_params,kn_sum1,5)
kn_sum_predictions=kn_sum.predict(X_test_sum)

scores_kn_sum=predictions_report(Y_test_sum,kn_sum_predictions)
full_time_kn_sum=round(time.time() - start_kn_sum,3)
print(full_time_kn_sum, 'c')

MAE = 438.908
RMSE = 550.915
Correlation coefficient = 0.170
8.682 c


In [14]:
output = output.append(pd.Series([str(kn_sum), scores_kn_sum[0],scores_kn_sum[1], 
                                  scores_kn_sum[2][0],full_time_kn_sum,'KNeighborsRegressor по сумме рейтингов'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986714,431.864722,0.458002,0.174,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.169,Обучение на разнице в рейтинге
2,"KNeighborsRegressor(algorithm='auto', leaf_siz...",438.908050,550.915478,0.170355,8.682,KNeighborsRegressor по сумме рейтингов


In [15]:
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_standard.xlsx',index=False)

In [16]:
def GridTune_diff(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train_diff2,Y_train_diff2)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [17]:
kn_diff_params =[{'n_neighbors':[3,5,9,11,15,20],
            'weights':['uniform','distance'],
            'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
            'metric':['euclidean','manhattan']}]

start_kn_diff = time.time()

kn_diff=KNeighborsRegressor(n_jobs=-1)
kn_diff_model=GridTune_diff(kn_diff_params,kn_diff,5)
kn_diff_predictions=kn_diff_model.predict(X_test_diff2)

scores_kn_diff=predictions_report(Y_test_diff2,kn_diff_predictions)
full_time_kn_diff=round(time.time() - start_kn_diff,3)
print(full_time_kn_diff, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'algorithm': 'auto', 'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}

Лучший алгоритм:
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
          metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
          weights='distance')
MAE = 173.281
RMSE = 220.950
Correlation coefficient = 0.394
220.721 c


In [18]:
start_kn_diff = time.time()

kn_diff=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
          metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
          weights='distance').fit(X_train_diff,Y_train_diff)

kn_diff_predictions=kn_diff.predict(X_test_diff)

scores_kn_diff=predictions_report(Y_test_diff,kn_diff_predictions)
full_time_kn_diff=round(time.time() - start_kn_diff,3)
print(full_time_kn_diff, 'c')

MAE = 178.464
RMSE = 226.658
Correlation coefficient = 0.413
11.059 c


In [19]:
output = output.append(pd.Series([str(kn_diff), scores_kn_diff[0],scores_kn_diff[1], 
                                  scores_kn_diff[2][0],full_time_kn_diff,'KNeighborsRegressor на разнице в рейтинге'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986714,431.864722,0.458002,0.174,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.169,Обучение на разнице в рейтинге
2,"KNeighborsRegressor(algorithm='auto', leaf_siz...",438.908050,550.915478,0.170355,8.682,KNeighborsRegressor по сумме рейтингов
3,"KNeighborsRegressor(algorithm='auto', leaf_siz...",178.463565,226.658384,0.413417,11.059,KNeighborsRegressor на разнице в рейтинге


In [20]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_standard.xlsx',index=False)

In [21]:
white_elos = (kn_sum_predictions + kn_diff_predictions)/2
black_elos = (kn_sum_predictions-kn_diff_predictions)/2
pred_elos=list(zip(white_elos,black_elos))
pred_elos[:5],elos[:5]

([(2179.087833553659, 2397.412166446341),
  (1551.8430104645454, 1715.62108258723),
  (2236.188248962285, 2303.79174605803),
  (2517.812701804186, 2574.0580141855744),
  (2163.4641765643123, 2075.0225904327986)],
    WhiteElo  BlackElo  MeanElos  DiffElos  SumElos
 0      2354      2411      2382       -57     4765
 1      1915      1999      1957       -84     3914
 2      2446      2191      2318       255     4637
 3      2168      2075      2121        93     4243
 4      2437      2254      2345       183     4691)

In [22]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
        count+=2
    return round(er/count,5)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
    return round(er,5)

In [23]:
find_abs_error(pred_elos,elos),find_mean_error(pred_elos,elos)

(126486.89375, 8.69087)